In [1]:
import importlib
import keys
importlib.reload(keys)
import json
import pprint
keychain = keys.keychain

pp = pprint.PrettyPrinter(indent=2)

client_id = keychain['youtube']['client_id']
scope = keychain['youtube']['scope']
redirect = keychain['youtube']['redirect_uris'][0]
print(client_id[:20], scope, redirect)

126920892433-e58m7pn ['https://www.googleapis.com/auth/youtube.readonly'] urn:ietf:wg:oauth:2.0:oob


In [2]:
from requests_oauthlib import OAuth2Session

In [3]:
session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)
auth_url, state = session.authorization_url(keychain['youtube']['auth_uri'])
print(auth_url)

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=2Z2i2bxsesCWa148cOHHWbb5Yhtdz8


In [4]:
import webbrowser
webbrowser.open(auth_url)

True

In [8]:
code = '4/o8Q6PHGs1BdxSr7h6cnNJIq6kS2_yP53WgIBiol2aLg'

In [9]:
token = session.fetch_token(keychain['youtube']['token_uri'], 
                                   code=code, 
                                   client_secret=keychain['youtube']['client_secret'])

In [ ]:
print(token)

In [10]:
importlib.reload(keys)
keychain = keys.keychain
keychain['youtube']

{'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
 'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
 'client_id': '126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com',
 'client_secret': 'flFfvozgUappY1MIRCPE5Cxc',
 'project_id': 'project5-187402',
 'redirect_uris': ['urn:ietf:wg:oauth:2.0:oob', 'http://localhost'],
 'scope': ['https://www.googleapis.com/auth/youtube.readonly'],
 'token_uri': 'https://accounts.google.com/o/oauth2/token'}

In [11]:
import requests
import pandas as pd
from xml.etree import ElementTree as ET
import io
D = {}
D['access_token'] = token['access_token']
url = 'https://www.googleapis.com/youtube/v3/channels?part=snippet&mine=true'
response = requests.get(url, params=D)
print(response)

<Response [200]>


In [76]:
url = 'https://www.googleapis.com/youtube/v3/videos'
D['regionCode'] = 'US'
D['chart'] = 'mostPopular'
D['part'] = ['snippet,contentDetails,statistics']
D['maxResults'] = '50'
popular_vid = requests.get(url, params = D)

popular_vid = popular_vid.json()
# popular_vid['nextPageToken']
popular_vid['nextPageToken']
D['pageToken'] = 'CDIQAA'
popular_vid = requests.get(url, params = D)
popular_vid = popular_vid.json()
popular_vid

{'etag': '"7991kDR-QPaa9r0pePmDjBEa2h8/V9kwKPlsucIDG-Mz3jGrCCYXhU8"',
 'items': [{'contentDetails': {'caption': 'false',
    'definition': 'hd',
    'dimension': '2d',
    'duration': 'PT1M15S',
    'licensedContent': True,
    'projection': 'rectangular'},
   'etag': '"7991kDR-QPaa9r0pePmDjBEa2h8/I_ytPz0jdt3aj39W4OcA4r77nf8"',
   'id': 'Cfn25A6krNg',
   'kind': 'youtube#video',
   'snippet': {'categoryId': '25',
    'channelId': 'UCqnbDFdCpuN8CMEg0VuEBqA',
    'channelTitle': 'The New York Times',
    'defaultAudioLanguage': 'en',
    'description': "President Trump has announced plans to reduce Bears Ears National Monument. President Obama had designated the 1.3 million acres in southern Utah at the request of Native American groups.\n\nRead the story here: http://nyti.ms/2BHSi5E\n\nMore from The New York Times Video: \nSubscribe: http://bit.ly/U8Ys7n\nWatch all of our videos here: http://nytimes.com/video\nFacebook: https://www.facebook.com/nytvideo\nTwitter: https://twitter.com/nyt

In [56]:
popular_vid['items']

data = {'id':[], 'categoryId':[], 'channelId':[], 'channelTitle':[], 'publishedAt':[],\
        'dislike':[],'like':[],'viewCount':[]}
for item in popular_vid['items']:
    data['id'].append(item['id'])
    data['categoryId'].append(item['snippet']['categoryId'])
    data['channelId'].append(item['snippet']['channelId'])
    data['channelTitle'].append(item['snippet']['channelTitle'])
    data['publishedAt'].append(item['snippet']['publishedAt'])
    data['dislike'].append(item['statistics']['dislikeCount'])
    data['like'].append(item['statistics']['likeCount'])
    data['viewCount'].append(item['statistics']['viewCount'])
df = pd.DataFrame(data)
df
# empty =[]
# for item in popular_vid['items']:
#     empty.append(item['statistics'])
# empty

,categoryId,channelId,channelTitle,dislike,id,like,publishedAt,viewCount
0,1,UCYUQQgogVeQY8cMQamhHJcg,CinemaSins,1955,pKkZjjhPCQM,43283,2017-12-05T16:58:28.000Z,836762
1,17,UCRijo3ddMTht_IHyNSNXpNQ,Dude Perfect,4758,-RmUADCWI4A,289860,2017-12-04T22:57:07.000Z,7558580
2,22,UCqwxMqUcL-XC3D9-fTP93Mg,LIL UZI VERT,6640,SxAp27sFaIM,114636,2017-12-04T22:42:37.000Z,1824114
3,24,UCMtFAi84ehTSYSE9XoHefig,The Late Show with Stephen Colbert,1542,YvcIeSfaB3c,12207,2017-12-05T08:35:01.000Z,696227
4,26,UCekQr9znsk2vWxBo3YiLq2w,You Suck At Cooking,395,a_iMlwkzwow,17333,2017-12-05T15:03:53.000Z,171077
5,24,UCLkZVEXz_qObWr7hAEbS_pQ,5MadMovieMakers,1208,me5ifeBFaFo,47544,2017-12-04T19:22:01.000Z,1381317
6,1,UCOpcACMWblDls9Z6GERVi1A,Screen Junkies,227,dDAlGl_8f9U,11896,2017-12-05T18:01:42.000Z,199240
7,17,UCDVYQ4Zhbm3S2dlz7P1GBDg,NFL,797,z1FfOwjlqxU,6586,2017-12-05T05:37:07.000Z,757281
8,23,UCPcFg7aBbaVzXoIKSNqwaww,jacksfilms,309,BOZngVYS04s,54606,2017-12-05T01:09:16.000Z,892070
9,24,UCq0OueAsdxH6b8nyAspwViw,Universal Pictures,4003,266EI7hdjFs,44959,2017-12-04T00:41:00.000Z,4106366


In [ ]:
popular_vid


In [ ]:
refresh_url = keychain['youtube']['token_uri']
token = session.refresh_token(refresh_url, 
                             client_id=keychain['youtube']['client_id'],
                             client_secret=keychain['youtube']['client_secret'])

In [ ]:
print(token)